In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

In [2]:
print(device)

cuda


In [6]:
chars = ""
with open("/content/alice_adventures.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)
print(vocab_size)


76


In [7]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class Head(nn.Module):
    #one head of self-attention

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        #input of size(batch, time-step, channels)
        #output of size (batch, time-step, head-size)
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #compute attention scores ("affinites")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        #perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    # Multiple heads of self-attention in parallel

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    #a simple linear layer followed by a non-linearity

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks  = nn.Sequential(*[Block(n_embd, n_head  = n_head)  for _ in range(n_layer)])

        self.ln_f  = nn.LayerNorm(n_embd)  #final  layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        # print(f"from index B: {B} T: {T}")

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            # print(f"B: {B} T: {T} C:{C}")
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :]
            #apply softmax to get probabilites
            probs = F.softmax(logits, dim=-1)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)

        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")

    xb, yb = get_batch('train')
    # print("xb: ",xb," yb: ",yb)

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.389971733093262, val loss: 4.39289665222168
step: 100, train loss: 2.301567316055298, val loss: 2.3019683361053467
step: 200, train loss: 1.7977758646011353, val loss: 1.8376173973083496
step: 300, train loss: 1.5431244373321533, val loss: 1.6483210325241089
step: 400, train loss: 1.387995958328247, val loss: 1.539805293083191
step: 500, train loss: 1.2714757919311523, val loss: 1.4877099990844727
step: 600, train loss: 1.1811829805374146, val loss: 1.45310378074646
step: 700, train loss: 1.097270131111145, val loss: 1.4322727918624878
step: 800, train loss: 1.0169620513916016, val loss: 1.4348037242889404
step: 900, train loss: 0.9505738615989685, val loss: 1.454553484916687
step: 1000, train loss: 0.8852418661117554, val loss: 1.4690896272659302
step: 1100, train loss: 0.8092447519302368, val loss: 1.5129281282424927
step: 1200, train loss: 0.7493802905082703, val loss: 1.5294424295425415
step: 1300, train loss: 0.6842765808105469, val loss: 1.57737135887146
st

In [9]:
# After training loop
import torch
# Save the model's state dictionary along with other necessary information
checkpoint = {
    'state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'vocab_size': vocab_size,
    'n_embd': n_embd,
    'block_size': block_size,
    'n_head': n_head,
    'n_layer': n_layer,
    'dropout': dropout,
    # Add other hyperparameters as needed
}

# Choose a path to save the checkpoint file
checkpoint_path = 'model_checkpoint.pth'

# Save the checkpoint
torch.save(checkpoint, checkpoint_path)

print("Model checkpoint saved.")


Model checkpoint saved.


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

In [11]:
# Load the checkpoint
checkpoint = torch.load("model_checkpoint.pth")

# Re-create the model architecture
loaded_model = GPTLanguageModel(checkpoint['vocab_size'])
loaded_model.load_state_dict(checkpoint['state_dict'])

# Move the loaded model to the desired device
loaded_model.to(device)

# If you want to resume training, load the optimizer state as well
optimizer = torch.optim.AdamW(loaded_model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

print("Model checkpoint loaded.")


Model checkpoint loaded.
